#Import Dataset

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import FactorAnalysis
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix as cm
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split, StratifiedKFold
from sklearn.feature_selection import RFE
import statsmodels.api as sm
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import matthews_corrcoef
from sklearn.preprocessing import LabelEncoder

In [3]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [4]:
import os

root_dir = "/content/gdrive/MyDrive/Colab Notebooks/2-1 패턴인식"

# Checking if our specified directory exists
os.path.exists(root_dir)

True

In [5]:
# Paths to the downloaded files
data_path = os.path.join(root_dir, "train.csv")

# Load data
df_train = pd.read_csv(data_path)

#make Dummy variables

In [6]:
columns = df_train.select_dtypes(include=[object]).columns
df = pd.concat([df_train, pd.get_dummies(df_train[columns])], axis=1)
df = df.drop(['job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week','poutcome', 'y'], axis =1)
print("\n","Describe Dataset_v2","\n")
print(df.info(),"\n \n \n")


 Describe Dataset_v2 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32950 entries, 0 to 32949
Data columns (total 65 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   id                             32950 non-null  int64  
 1   age                            32950 non-null  int64  
 2   campaign                       32950 non-null  int64  
 3   pdays                          32950 non-null  int64  
 4   previous                       32950 non-null  int64  
 5   emp.var.rate                   32950 non-null  float64
 6   cons.price.idx                 32950 non-null  float64
 7   cons.conf.idx                  32950 non-null  float64
 8   euribor3m                      32950 non-null  float64
 9   nr.employed                    32950 non-null  float64
 10  job_admin.                     32950 non-null  bool   
 11  job_blue-collar                32950 non-null  bool   
 12  job_entrepreneur      

#Scaling, Train-Test Split, Downsampling

In [7]:
min_max_scaler=preprocessing.MinMaxScaler()
data_scaled=pd.DataFrame(min_max_scaler.fit_transform(df),columns=df.columns)

In [8]:
y = data_scaled.y_yes
data_scaled = data_scaled.drop(['y_yes','y_no'], axis = 1)

In [9]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.under_sampling import TomekLinks

In [13]:
import numpy as np
from imblearn.under_sampling import TomekLinks

# 랜덤 시드 설정
np.random.seed(42)

from imblearn.under_sampling import TomekLinks

# TomekLinks를 사용하여 데이터 언더샘플링
X_samp, y_samp = TomekLinks().fit_resample(data_scaled, y)


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_samp, y_samp, shuffle=True, test_size=0.2, random_state=42)

#Training & Validation

In [15]:
!pip install catboost
from catboost import CatBoostClassifier, Pool, cv
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 2.4 MB/s eta 0:00:00


In [16]:
# CatBoost 모델 정의
model = CatBoostClassifier()

#model training
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# F1 점수 계산
f1 = f1_score(y_test, y_pred)
print("F1 Score:", f1)

Learning rate set to 0.040968
0:	learn: 0.6515366	total: 56.1ms	remaining: 56.1s
1:	learn: 0.6131901	total: 64.4ms	remaining: 32.1s
2:	learn: 0.5805840	total: 71.8ms	remaining: 23.9s
3:	learn: 0.5511228	total: 79.4ms	remaining: 19.8s
4:	learn: 0.5231459	total: 88.7ms	remaining: 17.6s
5:	learn: 0.4985285	total: 97.7ms	remaining: 16.2s
6:	learn: 0.4761764	total: 105ms	remaining: 15s
7:	learn: 0.4566759	total: 113ms	remaining: 14.1s
8:	learn: 0.4399647	total: 121ms	remaining: 13.3s
9:	learn: 0.4243230	total: 129ms	remaining: 12.8s
10:	learn: 0.4101962	total: 137ms	remaining: 12.3s
11:	learn: 0.3975197	total: 145ms	remaining: 12s
12:	learn: 0.3864814	total: 153ms	remaining: 11.6s
13:	learn: 0.3760451	total: 161ms	remaining: 11.3s
14:	learn: 0.3667604	total: 169ms	remaining: 11.1s
15:	learn: 0.3578082	total: 178ms	remaining: 10.9s
16:	learn: 0.3503376	total: 185ms	remaining: 10.7s
17:	learn: 0.3433546	total: 197ms	remaining: 10.8s
18:	learn: 0.3375935	total: 205ms	remaining: 10.6s
19:	learn

#Evaluation

In [17]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

y_prob = model.predict_proba(X_test)
print("Estimated probs:", y_prob[:10])

y_cls = model.predict(X_test)
print("Estimated classes:", y_cls[:10])
print()

Estimated probs: [[0.669285   0.330715  ]
 [0.96695478 0.03304522]
 [0.97948173 0.02051827]
 [0.6630125  0.3369875 ]
 [0.91190373 0.08809627]
 [0.94352645 0.05647355]
 [0.45047624 0.54952376]
 [0.97633464 0.02366536]
 [0.63555047 0.36444953]
 [0.95033321 0.04966679]]
Estimated classes: [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]



In [18]:
print("Accuracy:", accuracy_score(y_test, y_cls))
print("F1:", f1_score(y_test, y_cls))
print("ROC AUC:", roc_auc_score(y_test, y_prob[:, 1]))

Accuracy: 0.899652887346166
F1: 0.44308231173380036
ROC AUC: 0.8237624414835241


In [19]:
from sklearn.metrics import confusion_matrix

conf_matrix = confusion_matrix(y_test, y_cls)
conf_matrix_df = pd.DataFrame(
    conf_matrix,
    columns=["Predicted yes", "Predicted no"],
    index=["Actual yes", "Actual no"]
)
print(conf_matrix_df)

            Predicted yes  Predicted no
Actual yes           5449           108
Actual no             528           253


In [20]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_cls))

              precision    recall  f1-score   support

         0.0       0.91      0.98      0.94      5557
         1.0       0.70      0.32      0.44       781

    accuracy                           0.90      6338
   macro avg       0.81      0.65      0.69      6338
weighted avg       0.89      0.90      0.88      6338

